In [15]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0")
print(device)
torch.cuda.device_count()

cuda:0


1

In [16]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

24946


In [17]:
#Criando a Convolutional Neural Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #Conv2d - aplica uma convolução 2D
        #1º parâmetro - número de canais de entrada (RGB, BW)
        #2º parâmetro - número de canais de saida (quantos filtros serão usados)
        #3º parâmetro - kernel size (no nosso caso 5x5), lembrando, kernel size é o filtro que andará pela imagem
        self.conv1 = nn.Conv2d(1, 32, 5) 
        self.conv2 = nn.Conv2d(32, 64, 5) 
        self.conv3 = nn.Conv2d(64, 128, 5) 

        #Precisamos descobrir qual a saída da conv3, tem uma fórmula:
        #O = (W - K + 2P)/S + 1
        # O - tamanho da imagem de saída
        # W - Tamanho da imagem de entrada
        # K - Kernel Size 
        # P - Pooling
        # S - Stride
        #Mas vamos fazer por teste
        self._to_linear = None
        if self._to_linear is None:
            x = torch.randn(50,50).view(-1,1,50,50)
            #Função convs é pra descobrir o tamanho da saída da conv3
            self.convs(x)

        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)

    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        #Esse x[0].shape é pra gente ver o shape de saída
        print(x[0].shape)
        if self._to_linear is None:
            #x[0].shape[0] -> número de features
            #x[0].shape[1] -> largura de saída do conv3
            #x[0].shape[2] -> comprimento de saída do conv3
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        #Dando o flatten em conv3
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #O softmax retornará os valores entre 0 e 1. O dim fala a dimensão em qual vai ser aplicada o softmaxs
        return F.softmax(x, dim=1)

In [18]:
net = Net()
net.to(device)

import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

#Separando as imagens das suas classificações e setando ela como Numero de canais* Largura * Comprimento
#Sendo Largura=Comprimento = 50
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
#Realizamos essa divisão para deixarmos a matriz de imagens com valores entre 0 e 1 - Diminui custo computacional
X = X/255.0

y = torch.Tensor([i[1] for i in training_data])

#Aqui vamos setar a quantidade de fotos que utilizaremos para o validation data
VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

torch.Size([128, 2, 2])
2494


In [19]:
#X é vetor de 0 a 9, X[:-6] = 0,1,2,3
#X[-3:] = 3,4,5,6,7,8,9 
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [20]:
#Pegando número de fotos por vez
BATCH_SIZE = 100
EPOCHS = 7

for epochs in range(EPOCHS):
    #range - o primeiro é onde iremos começar
    #O segundo é até onde iremos
    # o terceiro é a quantidade por vez que o for passará, ou step sizes
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        #Aqui pegaremos as imagens de 100 em 100
        #Por exemplo: estaremos na imagem 100, o batch size pegará de 100 (i) até 200 (i+BATCH_SIZE); BATCH_S=100
        #O view é só para confirmarmos o número de features, número de canais, largura e comprimento
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i+BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        #Ao invés de pegar o net.zero_grad(), o optimizer tem controle dos parâmetros de net(), pois pega em net.parameters(). Mas pra que tu usa o optimizer ao invés do net?. No caso de ter uma bifurcação de modelo, com mais de uma neural network, o optimizer aparenta ser uma boa ideia, já que você pega os parâmetros de uma neural network em específico
        #optimizer.zero_grad()
        net.zero_grad()
        
        #Fazendo o predict com a rede neural para treinamento
        outputs = net(batch_X)

        #Pegando o erro da rede
        loss = loss_function(outputs, batch_y)
        #Backward vai pegar a variação de y em relação a x, para pegar o ângulo da função naquele ponto, se está em queda ou ascensão de y
        loss.backward()
        #Atualizando os weights da rede
        optimizer.step()

print(loss)

torch.Size([128, 2, 2])
 18%|█▊        | 40/225 [00:04<00:22,  8.31it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 19%|█▊        | 42/225 [00:04<00:21,  8.36it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 20%|█▉        | 44/225 [00:05<00:21,  8.38it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 20%|██        | 46/225 [00:05<00:21,  8.38it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 21%|██▏       | 48/225 [00:05<00:20,  8.45it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 22%|██▏       | 50/225 [00:05<00:20,  8.37it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 23%|██▎       | 52/225 [00:06<00:20,  8.40it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 24%|██▍       | 54/225 [00:06<00:20,  8.38it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 25%|██▍       | 56/225 [00:06<00:20,  8.34it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 26%|██▌       | 58/225 [00:06<00:19,  8.37it/s]torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
 27%|██▋       |

In [21]:
correct = 0
total = 0
#Para validar o nosso treinamento, não iremos precisar calcular nenhum gradiente, logo, com a função no_grad retiramos o cálculo de gradientes. Não queremos otimizar a Neural Network
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

([128, 2, 2])
 69%|██████▉   | 1726/2494 [00:06<00:02, 263.87it/s]
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2])
torch.Size([128, 2, 2